# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-09-28 03:44:49,361 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

### Solution

First, we create copies and aliased versions of the patents and citations tables. While doing this, we also separate out just the patent and postate fields from the patent table to make it easier to manage. We also set aside a full patents table to perform the final join

In [8]:
tc = citations.alias('tc')

In [9]:
tp = patents.alias('tp')
tp = tp.select(col("PATENT").alias("PCITING"), col("POSTATE").alias("CITING_POSTATE"))
tp1 = patents.alias('tp1')
tp2 = tp1.select(col("PATENT").alias("PCITED"), col("POSTATE").alias("CITED_POSTATE"))
tp4 = patents.alias("tp4")

We perform the first join on citing patent with our reduced patents dataframe

In [10]:
join1 = tc.join(tp, tc.CITING == tp.PCITING)
j1 = join1.alias('j1')

We perform the second join on cited patent with another copy of the reduced patents dataframe

In [11]:
join2 = j1.join(tp2, j1.CITED == tp2.PCITED)

Then we filter out the cases where the postate is not the same and also the cases where postate is null

In [12]:
j2 = join2.filter(join2.CITING_POSTATE == join2.CITED_POSTATE)
j3 = j2.filter(j2.CITING_POSTATE.isNotNull())

We arrange the dataframe removing the cited patents as this will simplify the counting process

In [13]:
j4 = j3.select(col("CITING"), col("CITING_POSTATE"), col("CITED_POSTATE"))

We then groupBy and count the number of occurrences. This statement will only give us the count and citing patent

In [14]:
j5 = j4.groupBy(col("CITING").alias("GCITING")).count()

We then join this information back into our citations table

In [15]:
j6 = j4.join(j5, j4.CITING == j5.GCITING)

We alias the count column and take only the distinct values

In [16]:
j7 = j6.select(col("CITING"), col("CITING_POSTATE"), col("CITED_POSTATE"), col("count").alias("CO_CITED_COUNT")).distinct()

We then join this back with the patent data

In [17]:
j8 = j7.join(tp4, j7.CITING == tp4.PATENT)

Dropping redundant patent column

In [18]:
j9 = j8.drop("PATENT")

Finally, we order the result by the aliased name for count in a descending order

In [19]:
j10 = j9.orderBy(col("CO_CITED_COUNT").desc())

In [20]:
j10.show(10)

2021-09-28 03:45:37,046 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------+-------------+--------------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITING|CITING_POSTATE|CITED_POSTATE|CO_CITED_COUNT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+--------------+-------------+--------------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|5959466|            CA|           CA|           125| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|
|5983822|            TX|           TX|           103| 1999|14564|   1998|     US|     TX|  5